# Ejercicio 6: Dense Retrieval e Introducción a FAISS

## Objetivo de la práctica

Generar embeddings con sentence-transformers (SBERT, E5), e indexar documentos con FAISS 

## Parte 0: Carga del Corpus
### Actividad

1. Carga el corpus 20 Newsgroups desde sklearn.datasets.fetch_20newsgroups.
2. Limita el corpus a los primeros 2000 documentos para facilitar el procesamiento.

In [1]:
# Importamos la función para obtener el dataset 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
#Cargamos el corpus completo (todos los subsets) y eliminamos headers, footers y quotes
newsgroups = fetch_20newsgroups(
    subset='all',
    remove=('headers', 'footers', 'quotes')
)
corpus = newsgroups.data
#Limitamos el corpus a los primeros 2000 documentos para facilitar el procesamiento
corpus = corpus[:2000]
# Verificamos cuántos documentos tenemos
print(f'Número de documentos en el corpus: {len(corpus)}')

Número de documentos en el corpus: 2000


## Parte 2: Generación de Embeddings
### Actividad

1. Usa dos modelos de sentence-transformers. Puedes usar: `'all-MiniLM-L6-v2'` (SBERT), o `'intfloat/e5-base'` (E5). Cuando uses E5, antepon `"passage: "` a cada documento antes de codificar.
2. Genera los vectores de embeddings para todos los documentos usando el modelo seleccionado.
3. Guarda los embeddings en un array de NumPy para su posterior indexación.

In [3]:
%pip install sentence-transformers -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Importamos las librerías necesarias
from sentence_transformers import SentenceTransformer
import numpy as np

# 1. Elegimos el modelo: SBERT o E5
model_name = 'all-MiniLM-L6-v2'      # SBERT
# model_name = 'intfloat/e5-base'    # E5 (recuerda añadir "passage: " a cada texto)

# Cargamos el modelo seleccionado
model = SentenceTransformer(model_name)

# 2. Preparamos los textos: si es E5, anteponemos "passage: " a cada documento
if 'e5' in model_name:
    inputs = [f'passage: {doc}' for doc in corpus]
else:
    inputs = corpus

# 3. Generamos los embeddings para todos los documentos
#    Ajusta batch_size si tu GPU/CPU lo requiere
embeddings_list = model.encode(
    inputs,
    batch_size=32,
    show_progress_bar=True
)

# 4. Convertimos la lista de embeddings a un array de NumPy
embeddings = np.array(embeddings_list)

# 5. (Opcional) Guardar el array en disco para uso posterior
np.save('embeddings.npy', embeddings)

# Verificamos la forma final: debería ser (2000, dimensión_del_modelo)
print('Shape de embeddings:', embeddings.shape)


C:\Users\roble\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\roble\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\roble\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.c

Shape de embeddings: (2000, 384)


## Parte 3: Indexación con FAISS
### Actividad

1. Crea un índice plano con faiss.IndexFlatL2 para búsquedas por distancia euclidiana.
2. Asegúrate de usar la dimensión correcta `(embedding_dim = doc_embeddings.shape[1])`.
3. Agrega los vectores de documentos al índice.

## Parte 4: Consulta Semántica
### Actividad

1. Escribe una consulta en lenguaje natural. Ejemplos:

    * "God, religion, and spirituality"
    * "space exploration"
    * "car maintenance"

2. Codifica la consulta utilizando el mismo modelo de embeddings. Cuando uses E5, antepon `"query: "` a la consulta.
3. Recupera los 5 documentos más relevantes con `index.search(...)`.
4. Muestra los textos de los documentos recuperados (puedes mostrar solo los primeros 500 caracteres de cada uno).